# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Personal/projects/wordle/valid_wordle_words.txt', sep=" ", header=None, names=["word"]) # the actual words that could be an answer
#df = pd.read_csv('/content/drive/MyDrive/Personal/projects/wordle/valid_wordle_guesses.txt', sep=" ", header=None, names=["word"]) # the words you can guess

In [ ]:
df

,word
0,aback
1,abase
2,abate
3,abbey
4,abbot
...,...
2310,young
2311,youth
2312,zebra
2313,zesty


In [ ]:
freq = {
    "E": 12.02,
    "T": 9.10,
    "A": 8.12,
    "O": 7.68,
    "I": 7.31,
    "N": 6.95,
    "S": 6.28,
    "R": 6.02,
    "H": 5.92,
    "D": 4.32,
    "L": 3.98,
    "U": 2.88,
    "C": 2.71,
    "M": 2.61,
    "F": 2.30,
    "Y": 2.11,
    "W": 2.09,
    "G": 2.03,
    "P": 1.82,
    "B": 1.49,
    "V": 1.11,
    "K": 0.69,
    "X": 0.17,
    "Q": 0.11,
    "J": 0.10,
    "Z": 0.07
}


def bestScore(potential):
  best_score = 0
  best_word = "temp"
  for word in potential:
    score = 0
    unique = list(set(word))
    for letter in unique:
      score += freq[letter.upper()]

    if score > best_score:
      best_score = score
      best_word = word

  return best_word

def getRandomWord(df):
  sampled_words = df.sample(n=30, random_state=42)["word"].tolist()
  return bestScore(sampled_words)

def findBestWord(df, sure, maybe, wrong, used, num_of_correct, correct_wrong_pos):
  potential = []
  for x in df.index:
    word = df.loc[x]["word"]

    if word in used:
      continue

    works = True
    for i in range(5):
      if word[i] != sure[i] and sure[i] != '':
        works = False

    if not works:
      continue

    for i in wrong:
      if i in word:
        works = False

    if not works:
      continue

    matches = {}

    for i in range(5):
      for j in maybe:
        if word[i] == j:
          try:
            matches[j] += 1
          except:
            matches[j] = 1

    if len(matches) < len(maybe):
      continue

    for i in matches:
      if matches[i] <= 0:
        works = False

    for i in num_of_correct.keys():
      temp_num_correct = 0
      for letter in word:
        if letter == i:
          temp_num_correct += 1

      if not temp_num_correct >= num_of_correct[i]:
        works = False
        break

    for i in correct_wrong_pos.keys():
      wrongs = correct_wrong_pos[i]
      for letter_id in range(5):
        if word[letter_id] == i and letter_id in wrongs:
          works = False

    if works:
      potential.append(word)
      if len(potential) > 30: # 30 words should be good enough
        break

  if len(potential) == 0:
    return False

  return bestScore(potential)

In [ ]:
sure = ['', '', '', '', '']
maybe = []
wrong = []
used = []
num_of_correct = {}
correct_wrong_pos = {}

def convertInputToUsable(letters, info):
  for i in range(5):
    if info[i] == 'g':
      sure[i] = letters[i]

      if letters[i] not in maybe:
        maybe.append(letters[i])

    elif info[i] == 'y':
      if info[i] not in correct_wrong_pos.keys():
        correct_wrong_pos[letters[i]] = [i]
      else:
        correct_wrong_pos[letters[i]].append(i)

      if letters[i] not in maybe:
        maybe.append(letters[i])

    elif info[i] == 'n':
      if letters[i] in maybe:
        count = 0
        for x in range(5):
          if letters[x] == letters[i] and (info[i] == 'g' or info[i] == 'y'):
            count += 1

        num_of_correct[letters[i]] = count
      else:
        wrong.append(letters[i])

def fullFind(dataframe, letters, info):
  convertInputToUsable(letters, info)
  dataframe = dataframe.sample(frac=1)
  return findBestWord(dataframe, sure, maybe, wrong, used, num_of_correct, correct_wrong_pos)

def resetInfo():
  sure = ['', '', '', '', '']
  maybe = []
  wrong = []
  used = []
  num_of_correct = {}
  correct_wrong_pos = {}

def doIt(starting_word="arise"):
  if len(starting_word) != 5:
    print("Please enter in a 5 letter word")

  if starting_word == "random":
    starting_word = getRandomWord(df)
  print("Next Suggested Input : " + starting_word)
  letters = starting_word
  used.append(starting_word)
  info = ""
  while info != True:
    info = input("Previous Info : ")
    if info == "reset" or info == "ggggg":
      break
    suggested = fullFind(df, letters, info)
    print("Next Suggested Input : " + suggested)
    used.append(suggested)
    letters = suggested
  print()
  print("now resetting ;)")
  resetInfo()

def doDouble(starting1 = "clint", starting2 = "soare"):
  if len(starting1) != 5 or len(starting2) != 5:
    print("Please enter in a 5 letter word")

  print("Next Suggested Input : " + starting1)
  letters = starting1
  used.append(starting1)
  info = ""
  info = input("Previous Info : ")
  temp = fullFind(df, letters, info)
  print("Next Suggested Input : " + starting2)
  letters = starting2
  used.append(starting2)

  while info != "reset":
    info = input("Previous Info : ")
    if info == "reset" or info == "ggggg":
      break
    suggested = fullFind(df, letters, info)
    print("Next Suggested Input : " + suggested)
    used.append(suggested)
    letters = suggested
  print()
  print("now resetting ;)")
  resetInfo()

In [ ]:
# # this is for debugging purposes
# print(sure)
# print(maybe)
# print(wrong)
# print(used)
# print(num_of_correct)
# print(correct_wrong_pos)

In [ ]:
resetInfo()

# Cool Stuff

This is a cool wordle bot I made! It takes in information about the game you are playing and then suggests an optimal word for you to input in next.

For each word that the bot suggests, input in 5 letters corresponding to whether or not the suggested word was correct (g for green, y for yellow, n for gray (nothing)), in the "Previous Info" Field


If wordle ended or you want to reset, input in "reset" for "Previous Info"

*Note: The default starting word is "arise". If you would like to change that, put a five letter word (in quotes) inside of the parentheses.*

The doDouble() function is the same as doIt but lets you choose a 2-word starting combo.

In [ ]:
#doIt()

In [ ]:
doDouble()

Next Suggested Input : clint
Previous Info : nnngn
Next Suggested Input : soare
Previous Info : nnyny
Next Suggested Input : hyena
Previous Info : nnggy
Next Suggested Input : amend
Previous Info : ggggg

now resetting ;)
